In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [4]:
#number of samples M and features N
M_tr = 153*5
N_tr = 180*200

def load_training(tr_path='./training_set/'):
    #matrix with training data
    tr_data = np.empty((M_tr, N_tr))
    #index of tr_data matrix
    m = 0
    for i in range(1,154):
        tgt = tr_path+str(i)+'/'
        pics = os.listdir(tgt)
        print i,
        print pics
        for pic in pics:
            tr_data[m] = cv.imread(tgt+pic, cv.IMREAD_GRAYSCALE).ravel()
            m +=1
    return tr_data

In [5]:
asdf = load_training()

1 ['9332898.4.jpg', '9332898.18.jpg', '9332898.13.jpg', '9332898.1.jpg', '9332898.16.jpg']
2 ['9336923.7.jpg', '9336923.14.jpg', '9336923.19.jpg', '9336923.5.jpg', '9336923.3.jpg']
3 ['9338446.17.jpg', '9338446.20.jpg', '9338446.3.jpg', '9338446.11.jpg', '9338446.10.jpg']
4 ['9338454.11.jpg', '9338454.16.jpg', '9338454.2.jpg', '9338454.1.jpg', '9338454.19.jpg']
5 ['9338462.6.jpg', '9338462.4.jpg', '9338462.12.jpg', '9338462.20.jpg', '9338462.17.jpg']
6 ['9338489.10.jpg', '9338489.9.jpg', '9338489.11.jpg', '9338489.2.jpg', '9338489.12.jpg']
7 ['9338497.1.jpg', '9338497.14.jpg', '9338497.20.jpg', '9338497.7.jpg', '9338497.18.jpg']
8 ['9338519.11.jpg', '9338519.9.jpg', '9338519.4.jpg', '9338519.16.jpg', '9338519.13.jpg']
9 ['9338527.7.jpg', '9338527.11.jpg', '9338527.20.jpg', '9338527.8.jpg', '9338527.1.jpg']
10 ['9338535.10.jpg', '9338535.15.jpg', '9338535.5.jpg', '9338535.17.jpg', '9338535.11.jpg']
11 ['9338543.5.jpg', '9338543.2.jpg', '9338543.18.jpg', '9338543.15.jpg', '9338543.10.jpg

AttributeError: 'NoneType' object has no attribute 'ravel'